# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Lab for basic text operation with Gemini and understanding Gemini fundamental.


## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 41.4 MB/s eta 0:00:00


## Authentication to access to the GCP & Google drive

In [3]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


## Set the environment on GCP Project

In [4]:
# Set the environment on GCP Project
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL_NAME="gemini-1.5-flash"

## Vertex AI initialization


In [5]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

# Initialize the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION) # https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

system_instructions = [
 "Respond in Korean",
 "Respond in structured ways"
]

# Access to the generative model.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

## Function to get the response

In [6]:
def generate(query:str, stream:bool):
    """
    Generate a response from the model.
    query :query to be sent to the model
    Returns:The generated response.
    """

    # Request body : https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    # parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
    )

    return responses



## Run example

In [8]:

from time import perf_counter

t1_start = perf_counter()

prompt = """
          You are a Generative AI specialist to help people understand the concept of AI.
          Explain what's the pros and cons of using AI in the following instructions.

          1. Define what's the Generative AI.
          2. What are differences between typical machine learning and Generative AI.
          3. What are the pros and cons of using Generative AI.
          4. Explain use cases of Generative AI in the manufacturing industry.
"""
stream = False

responses = generate(prompt, stream)

if stream:
  for response in responses:
    print(response.text, end="")
else:
  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")

  display(Markdown(responses.text))

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")



Token Count : total_tokens: 105
total_billable_characters: 363


 Usage metadata:
{'prompt_token_count': 105, 'candidates_token_count': 735, 'total_token_count': 840}

 Finish reason:
1

 Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.0279541016
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.106933594
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.0229492188
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.12890625
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.103515625
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0815429688
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.0311279297
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.068359375
]


## 생성형 AI에 대한 이해: 장단점과 제조업 적용 사례

**1. 생성형 AI란 무엇인가요?**

생성형 AI는 새로운 콘텐츠를 생성하는 데 특화된 인공지능 기술입니다. 텍스트, 이미지, 음악, 비디오, 코드 등 다양한 형태의 콘텐츠를 만들어낼 수 있으며, 기존 데이터를 학습하여 새로운 데이터를 생성하는 능력을 갖추고 있습니다.

**2. 일반적인 머신러닝과 생성형 AI의 차이점은 무엇인가요?**

| 특징 | 일반적인 머신러닝 | 생성형 AI |
|---|---|---|
| 목표 | 입력 데이터를 기반으로 예측 또는 분류 | 새로운 데이터를 생성 |
| 예시 | 이미지 분류, 텍스트 감정 분석 | 이미지 생성, 텍스트 작성 |
| 학습 방식 | 기존 데이터에서 패턴을 학습 | 기존 데이터에서 분포를 학습 |

**3. 생성형 AI를 사용하는 장점과 단점은 무엇인가요?**

**장점:**

* **창의성 증진:** 새로운 아이디어와 디자인을 생성하여 창의성을 높일 수 있습니다.
* **효율성 향상:** 반복적인 작업을 자동화하여 효율성을 높일 수 있습니다.
* **개인화된 경험 제공:** 사용자의 선호도에 맞는 콘텐츠를 생성하여 맞춤형 경험을 제공할 수 있습니다.
* **데이터 부족 문제 해결:** 데이터가 부족한 경우에도 새로운 데이터를 생성하여 모델 학습에 활용할 수 있습니다.

**단점:**

* **데이터 편향:** 학습 데이터에 편향이 존재하면 생성된 콘텐츠에도 편향이 반영될 수 있습니다.
* **저작권 문제:** 생성된 콘텐츠의 저작권 문제가 발생할 수 있습니다.
* **사용자의 의도 오해:** 사용자의 의도를 정확하게 이해하지 못하고 예상치 못한 결과를 생성할 수 있습니다.
* **윤리적 문제:** 생성된 콘텐츠의 윤리적 문제 발생 가능성이 있습니다.

**4. 제조업에서의 생성형 AI 활용 사례는 무엇인가요?**

* **제품 설계:** 3D 모델링을 통해 새로운 제품 디자인을 생성하고 시뮬레이션을 통해 최적화된 설계를 도출할 수 있습니다.
* **생산 공정 최적화:** 생산 공정의 효율성을 높이기 위한 새로운 공정 방식을 제안하고 시뮬레이션을 통해 검증할 수 있습니다.
* **품질 관리:** 제품의 불량률을 줄이기 위한 새로운 품질 검사 방법을 개발하고 시뮬레이션을 통해 검증할 수 있습니다.
* **예측 유지보수:** 기계 고장을 예측하고 사전에 예방 조치를 취함으로써 생산 중단 시간을 줄일 수 있습니다.


생성형 AI는 다양한 분야에서 혁신을 가져올 잠재력을 가지고 있지만, 윤리적 문제와 데이터 편향 등의 문제점을 해결해야 하는 과제도 존재합니다. 


Time : 4.362509865999982 seconds


